In [1]:
!pip install --upgrade langchain
!pip install openai
!pip install pinecone-client
!pip install tiktoken -q
!pip install uuid
!pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.2 MB/s eta 0:00:00


In [10]:
import os
import openai
import pinecone
import cohere
import tiktoken
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from openai import OpenAI
from uuid import uuid4

# Set OpenAI API key in the environment variable
os.environ["OPENAI_API_KEY"] = ""

# Initialize OpenAI client
client = OpenAI()

# Initialize tokenizer for counting tokens in a text
tokenizer = tiktoken.get_encoding("cl100k_base")

# Set up Pinecone
pinecone.init(api_key="", environment="gcp-starter")

# Define Pinecone index name
index_name = "test"

# Create Pinecone index instance
pinecone_index_instance = pinecone.Index(index_name)

# Initialize Cohere client
cohere_instance = cohere.Client("")

# Define embedding mdoel name
embedding_engine_name = "text-embedding-ada-002"

# Define GPT model name
gpt_model_name = "gpt-3.5-turbo"

# Set temperature for GPT responses (a higher value makes output more random)
temperature = 0.4

# Add detailed comments explaining each section of the code
# ...

# Note: The code is incomplete, and I've added comments based on the provided snippets.
# Please make sure to complete the cod

In [11]:
def token_len(text):
    """
    Calculates the number of tokens in a given text using the specified tokenizer.

    Parameters:
    - text (str): The input text to calculate token length for.

    Returns:
    - int: The number of tokens in the given text.
    """
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)


def split_docs(documents, chunk_size=400, chunk_overlap=20):
    """
    Splits a list of documents into chunks based on the specified parameters.

    Parameters:
    - documents (list): List of documents (text) to be split.
    - chunk_size (int): Size of each chunk in tokens.
    - chunk_overlap (int): Overlap between chunks in tokens.

    Returns:
    - list: List of split documents.
    """
    # Initialize text splitter with the specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=token_len,
        separators=["\n\n", "\n", " ", ""],
    )

    # Split the documents using the text

In [44]:
metadata = []


def create_metadata(doc):
    """
    Create metadata for each chunk in a document.

    Parameters:
    - doc (list): List of document chunks.

    Returns:
    - None
    """
    for i, chunk in enumerate(doc):
        metadata.append(
            {
                "chunk": i,
                "text": chunk,
            }
        )


def insert_data_to_pinecone(text):
    """
    Insert data into Pinecone index.

    Parameters:
    - text (str): The input text.

    Returns:
    - None
    """
    print("aa")

    # Split the text into chunks
    docs = split_docs(text)
    print(docs)

    # Generate embeddings for the document chunks
    embeddings = client.embeddings.create(input=docs, model=embedding_engine_name)

    # Extract the embedding vectors and generate UUIDs for each vector
    embedding_vectors = [embeddings.data[0].embedding]
    ids = [str(uuid4()) for _ in range(len(embedding_vectors))]

    # Create metadata for the document chunks
    create_metadata(docs)

    # Insert embeddings and metadata into the Pinecone index
    pinecone_index_instance.upsert(vectors=zip(ids, embedding_vectors, metadata))


# Note: The comments provide an overview of each function and its purpose.
# Please adjust or expand the comments based on the specific implementation details if needed.

In [24]:
data=f'''Shobhit Vishnoi
To work in a learning and challenging environment , utilizing my skill and knowledge to the best of my abilities and contribute positively to my personal growth as well as growth of the organization.
EXPERIENCE
RapidInnovation, Noida — Blockchain Lead (09/ 2020 - PRESENT) '''

In [45]:
insert_data_to_pinecone(data)

aa
['Shobhit Vishnoi\nTo work in a learning and challenging environment , utilizing my skill and knowledge to the best of my abilities and contribute positively to my personal growth as well as growth of the organization.\nEXPERIENCE\nRapidInnovation, Noida — Blockchain Lead (09/ 2020 - PRESENT)']
CreateEmbeddingResponse(data=[Embedding(embedding=[-0.0035368851386010647, -0.034776534885168076, -0.01530494261533022, -0.012785890139639378, -0.028894206508994102, 0.016584893688559532, -0.02306634560227394, 0.014365403912961483, -0.04093119129538536, -0.0036662418860942125, 0.03213493153452873, -0.008918805047869682, 0.0239922683686018, -0.006971646565943956, 0.003395613981410861, -0.0072848256677389145, 0.020560910925269127, -0.011219993233680725, 0.004898534622043371, -0.007924800738692284, -0.03594755008816719, 0.007237168028950691, 0.014079458080232143, -0.0023948014713823795, -0.004381108097732067, -0.010804690420627594, 0.02561262995004654, -0.0192128773778677, 0.008251597173511982, 

In [52]:
def query_pinecone_with_filter(
    query, top_k_pinecone_results=20, top_k_rerank_results=5
):
    # Create embeddings for the query
    response = client.embeddings.create(input=query, model=embedding_engine_name)

    # Extract the query embeddings
    query_embeddings = response.data[0].embedding

    # Query the Pinecone index with the query embeddings and filter by user
    pinecone_result = pinecone_index_instance.query(
        [query_embeddings], top_k=top_k_pinecone_results, include_metadata=True
    )

    # Create a dictionary to map document text to its index in the Pinecone result
    docs = {x["metadata"]["text"]: i for i, x in enumerate(pinecone_result["matches"])}

    # Rerank the documents using Cohere
    rerank_docs = cohere_instance.rerank(
        query=query,
        documents=docs.keys(),
        top_n=top_k_rerank_results,
        model="rerank-english-v2.0",
    )

    sources = {"results": []}
    texts = ""

    # Process reranked documents
    if rerank_docs:
        for i, doc in enumerate(rerank_docs):
            # Filter documents based on relevance score
            if doc.relevance_score > 0.5:
                index_of_pinecone = doc.index
                if index_of_pinecone is not None and 0 <= index_of_pinecone < len(
                    pinecone_result["matches"]
                ):
                    specific_result = pinecone_result["matches"][index_of_pinecone]
                    result_dict = {
                        "text": specific_result["metadata"]["text"],
                    }
                    sources["results"].append(result_dict)
                else:
                    print(f"Index {index_of_pinecone} is out of range.")
                texts += doc.document["text"]

    return texts, sources


# Note: The comments provide an overview of each section and its purpose.
# Please adjust or expand the comments based on the specific implementation details if needed.

In [61]:
context = []


def generate_LLM_response(query, texts):
    """
    Generate a response using a language model based on a user query and knowledge base texts.

    Parameters:
    - query (str): User's query.
    - texts (str): Knowledge base texts.

    Returns:
    - str: Generated response from the language model.
    """
    # Construct the assistant's response template
    template = f"""
        Role: You are a helpful assistant specialized in providing answers when provided with the knowledge base.

        1. Search for an answer for the query in the information from the knowledge base.
        4. If the answer is unavailable, rely on your own knowledge.
        5. Maintain the tone and language style observed in the knowledge base.
        6. Never break character.
        Query: {query}

        Knowledge base: {texts}
    """

    # Prepare the conversation messages
    messages = (
        [
            {
                "role": "system",
                "content": "You are an assistant. Always limit your response to 180-200 words only. Never exceed this limit.",
            },
        ]
        + context
        + [{"role": "user", "content": template}]
    )

    # Generate a response using the GPT model
    res = client.chat.completions.create(
        model=gpt_model_name,
        messages=messages,
        temperature=temperature,
    )

    # Extract the response from the GPT model and update the conversation context
    response = res.choices[0].message.content
    context.append({"role": "user", "content": query})
    context.append({"role": "assistant", "content": response})

    return response


# Note: The comments provide an overview of each section and its purpose.
# Please adjust or expand the comments based on the specific implementation details if needed.

In [ ]:
while True:
    query = input("Please provide the query (type 'exit' to quit): ")

    if query.lower() == 'exit':
        break  # exit the loop if the user enters 'exit'

    texts = query_pinecone_with_filter(query)
    print(generate_LLM_response(query, texts))